In [3]:
%run "../../PyUtils/common.ipynb"
import math;
import requests as rq;
from datetime import datetime;

patient_data = LoadDataSet("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20patient_jlee", checkForDateTime=False);
patient_data[:]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,pname,gender,age,height,weight,pef_normal,pef_gy_bound,pef_yr_bound,climate_loc,pollution_loc,pefuse_count,pefuse_start,pefuse_end
0,hand2,f,63,154,71,370,296,185,119,1,658,2015-01-01,2016-10-19
1,kimj,f,66,150,57,355,284,176,119,2,365,2015-02-05,2016-02-04
2,kuky,m,48,165,65,590,472,295,119,3,439,2014-11-21,2016-02-02
3,leeh2,f,61,159,63,385,308,192,112,4,592,2015-03-10,2016-10-21
4,kwonc,m,45,172,95,640,512,320,112,4,0,null,null
5,kimk,f,37,160,55,400,320,200,112,4,0,null,null
6,uhy,f,35,149,58,375,300,188,112,5,690,2014-01-01,2015-11-21


In [4]:

print(sum(patient_data.pefuse_count[:])*2-690)
#pef_data = LoadDataSet("http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=SELECT%20*%20FROM%20pef_jlee%20WHERE%20pname='" + str(patient_data.pname[0]) + "'%20ORDER%20BY%20ampm,%20date,%20time", checkForDateTime=False);
#print(len(pef_data))
#pef_data[709:]

4798


In [87]:
url = "http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=";
ampm = ['am', 'pm'];
pollutant_sum = ['so2_sum', 'co_sum', 'o3_sum', 'no2_sum', 'pm10_sum'];


for p_num in range(0, len(patient_data)) :
    if patient_data.pefuse_count[p_num] != 0 :
        for j in ampm : 
            idx = 7;
            calculated_data = LoadDataSet(url + "SELECT%20*%20FROM%20calculated_jlee%20WHERE%20pname='" + str(patient_data.pname[p_num]) + "'%20and%20ampm='" +j+ "'%20ORDER%20BY%20date,%20time", checkForDateTime=False);

            if str(patient_data.pname[p_num]) == 'uhy' and j == 'pm' :
                continue;
            
            calculated_data = calculated_data.drop(['pname','date','ampm','time','pef_zone','tm_window','so2_avg','co_avg','o3_avg','no2_avg','pm10_avg','sealevelpressurehpa','temperature_diff','airpressure_diff','precipitation_diff'], axis=1);
            calculated_data.rename(columns={'so2_sum': 'so2_sum1', 'co_sum': 'co_sum1', 'o3_sum': 'o3_sum1', 'no2_sum': 'no2_sum1', 'pm10_sum': 'pm10_sum1'}, inplace=True);
            #print(calculated_data)
            
            for pollutant in pollutant_sum :
                new_col2 = list();
                new_col3 = list();
                new_col2.append(0); new_col2.append(0);
                new_col3.append(0); new_col3.append(0);

                for i in range(2, len(calculated_data.index)) :
                    colname = str(pollutant+'1');
                    #print(colname)
                    new_col2.append(calculated_data.loc[i][colname] + calculated_data.loc[i-1][colname])
                    new_col3.append(calculated_data.loc[i][colname] + calculated_data.loc[i-1][colname] + calculated_data.loc[i-2][colname]);
                
                calculated_data.insert(loc=idx, column=pollutant+'2', value=new_col2);
                calculated_data.insert(loc=idx+1, column=pollutant+'3', value=new_col3);
                
                idx = idx+3;
                    
            calculated_data = calculated_data[2:]
            print(calculated_data);
            calculated_data.to_csv("./calculated_3/"+str(patient_data.pname[p_num])+"_" + j + "_calculated_3.csv", sep = ',', index = False);

     pef  so2_max  co_max  o3_max  no2_max  pm10_max  so2_sum1  so2_sum2  so2_sum3  co_sum1  co_sum2  co_sum3  o3_sum1  o3_sum2  o3_sum3  no2_sum1  no2_sum2  no2_sum3  pm10_sum1  pm10_sum2  pm10_sum3  temperaturec  windspeedms  winddirection  precipitationpercent  vaporpressurehpa  dewpointtemperaturec  airpressurehpa  groundtemperaturec  temperature_max  airpressure_max  precipitation_max  temperature_min  airpressure_min  precipitation_min
2    260    0.013   1.000   0.015    0.039    52.000     0.174     0.299     0.378   16.975   34.020   46.810    0.180    0.479    0.907     0.797     1.381     1.681    857.550   1743.833   2734.133         0.700        1.600         70.000                67.000             4.300                -4.700        1013.600              -0.700            1.900         1023.400             71.000           -9.600         1013.200             38.000
3    273    0.009   1.300   0.021    0.065    75.000     0.137     0.311     0.436   22.300   39.275   56.32